In [63]:
import pandas as pd
import numpy as np
df = pd.read_csv("/Users/cpreeldumas/Documents/GitHub/MLC_SP23/clean_data/merged_data_v3.csv", low_memory=False, index_col=[0])
df.head()


,bbl,total_units_19,year_built,ejectment_count,possession_count,eviction_count,aleration_count,altered_lessthan5yrs,altered_morethan5yrs,classanet_avg,...,boro_3,boro_4,boro_5,rs_share_diff_new,hpd_call_2017_19,hh_inc_avg_19_log1,pop_race_asian_pct_19_log1,pop_race_black_pct_19_log1,pop16_unemp_pct_19_log1,pop_mobile1_diff_state_pct_19_log1
0,1000780047,483,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,4.0,12.276639,0.150551,0.018246,0.035008,0.074674
1,1001350019,4,1915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0.0,12.897523,0.096864,0.055298,0.002253,0.014341
2,1001620013,20,1915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1.0,2.0,10.973689,0.461379,0.096629,0.037102,0.008935
3,1001630024,16,1915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,1.0,10.973689,0.461379,0.096629,0.037102,0.008935
4,1001630033,11,1915,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0.0,0.0,10.973689,0.461379,0.096629,0.037102,0.008935


# Train-test split

In [64]:
from sklearn.model_selection import train_test_split

X = df.drop(['rs_share_diff_new'], axis = 1)
y = df['rs_share_diff_new']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Plain Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, accuracy_score 

logreg = LogisticRegression()

# Training
logreg .fit(X_train, y_train)

# Predict
y_pred = logreg.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test,y_pred)

# AUC
y_pred_proba = logreg.predict_proba(X_test)[:,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

# Printing the results
print("Accuracy is",accuracy)
print("AUC is", auc)

Accuracy is 0.6660849928146171
AUC is 0.3570055442024259


# GridSearchCV with Ridge & Lasso

In [67]:
# scaling to have mean = 0 and std = 1
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

scoring = {'Accuracy':'accuracy',
           'F1 Score':'f1',
           'Precision':'precision',
           'Recall':'recall',
           'AUC':'roc_auc'}

# l1 = lasso l2 = ridge
parameters = [{'penalty':['l1','l2']},  
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]


logreg = LogisticRegression()
clf = GridSearchCV(logreg,                   
                   param_grid = parameters,  
                   scoring= scoring,  
                   refit = "AUC",
                   cv=5)                    


# Training
fit = clf.fit(X_train,y_train)

print("Tuned Hyperparameters :", clf.best_params_) # why is there no best param for penalty? 
print("Best Score :", clf.best_score_)

Tuned Hyperparameters : {'C': 0.1}
Best Score : 0.7010877084298297


In [69]:
# All CV metrics for best model

best_params_index = fit.cv_results_['rank_test_AUC'][0] - 1
print("Best model AUC:", fit.cv_results_['mean_test_AUC'][best_params_index])
print("Best model accuracy:", fit.cv_results_['mean_test_Accuracy'][best_params_index])
print("Best model F1 score:", fit.cv_results_['mean_test_F1 Score'][best_params_index])
print("Best model precision:", fit.cv_results_['mean_test_Precision'][best_params_index])
print("Best model recall:", fit.cv_results_['mean_test_Recall'][best_params_index])

Best model AUC: 0.7010179931719092
Best model accuracy: 0.7162703165098374
Best model F1 score: 0.48292833410845165
Best model precision: 0.6372527282144532
Best model recall: 0.38889500745321814


In [70]:
# test set
from sklearn import metrics

y_hat_test = fit.predict(X_test) 
auc = metrics.roc_auc_score(y_test, y_hat_test)
acc = metrics.accuracy_score(y_test, y_hat_test)
f1 = metrics.f1_score(y_test, y_hat_test)
prec = metrics.precision_score(y_test, y_hat_test)
rec = metrics.recall_score(y_test, y_hat_test)

print("Out of sample AUC is:", auc)
print("Out of sample accuracy is:", acc)
print("Out of sample F1 score is:", f1)
print("Out of sample precision is:", prec)
print("Out of sample recall is:", rec)

Out of sample AUC is: 0.6344172883648022
Out of sample accuracy is: 0.7175118045575857
Out of sample F1 score is: 0.476009139375476
Out of sample precision is: 0.6253126563281641
Out of sample recall is: 0.38426068244697204


In [74]:
logreg = LogisticRegression(C = 0.1)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print("Accuracy:",logreg.score(X_test, y_test))

Accuracy: 0.7175118045575857


# Tuning History

In [ ]:
# Without scaling, same parameters

# Out of sample AUC is: 0.5
# Out of sample accuracy is: 0.6660849928146171
# Out of sample F1 score is: 0.0
# Out of sample precision is: 0.0
# Out of sample recall is: 0.0